In [7]:
#Aqui subire las consultas que hice en la base de datos en BigQuery sobre test hechos a nivel nacional en EE.UU con sus respectivos positivos y negativos, hospitalizaciones, gente con ventilador, muertes, entre otros.

import pandas as pd

# Ruta de acceso de los datos obtenidos de la base de datos, en formato JSON.
ruta_archivo = "C:\\Users\\nicol\\Downloads\\bquxjob_67693a0_18ec56ddbfe.json"

# Aquí cargo los datos desde el archivo JSON en un DataFrame
df = pd.read_json(ruta_archivo)

# Mostrar las primeras filas del DataFrame
print(df.head())

      death       date  hospitalized_currently  hospitalized_increase  \
0  515151.0 2021-03-07                 40199.0                    726   
1  514309.0 2021-03-06                 41401.0                    503   
2  512629.0 2021-03-05                 42541.0                   2781   
3  510408.0 2021-03-04                 44172.0                   1530   
4  508665.0 2021-03-03                 45462.0                   2172   

   hospitalized_cumulative  in_icu_currently  in_icu_cumulative    negative  \
0                 776361.0            8134.0            45475.0  74582825.0   
1                 775635.0            8409.0            45453.0  74450990.0   
2                 775132.0            8634.0            45373.0  74307155.0   
3                 772351.0            8970.0            45293.0  74035238.0   
4                 770821.0            9359.0            45214.0  73857281.0   

   death_increase    positive  total_test_results  on_ventilator_currently  
0        

In [8]:
#2222